In [ ]:
# conda install python=3.9 scikit-learn=1.6.1

In [1]:
## local machine
src_path = '../src/'
lib_path = '../lib/'
input_path = '../../kaggle/input/home-credit-default-risk/'

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)

pd.set_option("display.max_columns", 200)

import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import (
    ColumnTransformer,
    make_column_selector,
    make_column_transformer,
)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import (
    LogisticRegression,
    LogisticRegressionCV,
    RidgeClassifier,
    RidgeClassifierCV,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    roc_auc_score,
    make_scorer,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import (
    RandomForestClassifier,
    HistGradientBoostingClassifier,
    StackingClassifier,
    AdaBoostClassifier,
)
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV

from sklearn import set_config

set_config(transform_output="pandas")

from category_encoders import HashingEncoder


# import joblib
# saved = joblib.load(lib_path + "data_processing.joblib")
# preprocessor = saved["preprocessor"]
# non_co_cols = saved["non_co_cols"]
# merged_application_test = saved["merged_application_train"]

# /kaggle/input/preprocessor/data_processing.joblib



import joblib
data_processing = joblib.load(lib_path + "data_processing.joblib")
preprocessor = data_processing["preprocessor"]
non_co_cols = data_processing["non_co_cols"]
merged_application_test = data_processing["merged_application_train"]

# import sys
# sys.path.insert(0, src_path)
# from eval_classification import eval_classification
# del sys.path[0]

# /kaggle/input/github-import/kaggle/src/eval_classification.py

import sys

sys.path.insert(0, src_path)
from eval_classification import eval_classification

del sys.path[0]

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# separate data
X = merged_application_test.drop(columns=["TARGET", "SK_ID_CURR"])
y = merged_application_test["TARGET"].copy()
# split data 80/20 for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)
# fit data on preprocessor
preprocessor.fit(X_train, y_train)
# transform data
X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)
# drop columns with collinear relationships (Pearson's correlation coefficients > 0.8)
X_train.drop(columns=non_co_cols, inplace=True)
X_test.drop(columns=non_co_cols, inplace=True)

In [ ]:

df = pd.read_csv("kc_house_data.csv")
df
df.info()
df = df.drop(columns=["id", "date"])
df.info()
df.describe()
df.duplicated().sum()
df = df.drop_duplicates()
df.duplicated().sum()
X = df.drop(columns=["price"])
y = df["price"].copy()
X.shape
input_shape = X.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)
def create_model():
    model = Sequential()
    model.add(Dense(16, activation="relu", input_shape=(input_shape,)))
    model.add(Dense(1, activation="linear"))


    model.compile(optimizer="adam", loss="mse", metrics=["mae"])

    return model

model = create_model()
model.summary()
input_shape
history = model.fit(
    X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=100
)
history.history.keys()
history.history["loss"]
plt.plot(history.history["mae"], label="train_mae")
plt.plot(history.history["val_mae"], label="val_mae")
history2 = model.fit(
    X_train_scaled, y_train, validation_data=(X_val_scaled, y_val), epochs=100
)
mae = history.history["mae"] + history2.history["mae"]
val_mae = history.history["val_mae"] + history2.history["val_mae"]

plt.plot(mae, label="train_mae")
plt.plot(val_mae, label="val_mae")
plt.legend()
def create_model2():
    model = Sequential()
    model.add(Dense(16, activation="relu", input_shape=(input_shape,)))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="linear"))


    model.compile(optimizer="adam", loss="mse", metrics=["mae"])

    return model

model2 = create_model2()
model2.summary()
history_model2 = model.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=250,
    batch_size=256,
)
plt.plot(history_model2.history["mae"], label="train_mae")
plt.plot(history_model2.history["val_mae"], label="val_mae")
history_model2_2 = model.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=250,
    batch_size=256,
)
mae_2 = history_model2.history["mae"] + history_model2_2.history["mae"]
val_mae_2 = history_model2.history["val_mae"] + history_model2_2.history["val_mae"]

plt.plot(mae_2, label="train_mae")
plt.plot(val_mae_2, label="val_mae")
plt.legend()
mnist = pd.read_csv("mnist.csv")
mnist.head()
X = mnist.drop(columns=["label"])
y = mnist["label"].copy()

y.info()
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

y_enc = ohe.fit_transform(y.values.reshape(-1, 1))
y_enc
ohe.categories_
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape
y_train_enc = y_enc[y_train.index]
y_test_enc = y_enc[y_test.index]
y_val_enc = y_enc[y_val.index]
X_train.min().min()
X_train.max().max()
X_train /= 255
X_val /= 255
X_test /= 255
y.nunique()
from tensorflow.keras.layers import Input
def make_class_model():
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(255, activation="relu"))
    model.add(Dense(128, activation="relu"))
    model.add(Dense(y.nunique(), activation="softmax"))

    model.compile(
        optimizer="adam", 
        # loss="sparse_categorical_crossentropy", # not one hot encoded
        loss="categorical_crossentropy",  # one hot encoded
        metrics=["accuracy"]
    )
    return model
model = make_class_model()
history = model.fit(
    X_train, 
    y_train_enc, 
    validation_data=(X_val, y_val_enc), 
    epochs=50, 
    batch_size=256
)
y_pred = model.predict(X_val)
y_pred.shape
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
y_pred_flat = np.argmax(y_pred, axis=1)
y_pred_flat.shape
print(classification_report(y_val, y_pred_flat))
ConfusionMatrixDisplay.from_predictions(y_val, y_pred_flat)
decoder = dict(zip(range(y.nunique()), ohe.categories_[0]))
decoder
def plot_history(history):
    for metric in history.history.keys():
        if not metric.startswith("val"):
            plt.plot(history.history[metric], label=metric)
            if f"val_{metric}" in history.history.keys():
                plt.plot(history.history[f"val_{metric}"], label=f"val_{metric}")
            plt.title(metric)
            plt.legend()
            plt.show()

plot_history(history)
model.save("mnist_model1.keras")
from tensorflow.keras.models import load_model
loaded_model = load_model("mnist_model1.keras")
loaded_model.summary()
loaded_model.predict(X_test)